<a href="https://www.kaggle.com/code/manepallimohankumar/oct-3to2-tl?scriptVersionId=164582108" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install keract

In [2]:
import os
from glob import glob
import pandas as pd
import numpy as np
from numpy import expand_dims
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sn
from skimage.transform import resize
from skimage.color import gray2rgb
from sklearn.metrics import classification_report, confusion_matrix
from IPython.display import SVG
# import keract
from tensorflow import keras
from tensorflow.keras import applications, optimizers
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.utils import to_categorical, model_to_dot, plot_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau

2024-02-27 18:06:41.266116: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-27 18:06:41.266213: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-27 18:06:41.396719: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
img_width, img_height = 150, 150 
channels = 3
batch_size = 32

In [4]:
train_path="/kaggle/input/oct-classes/OCT2Classes/OCT_2_Classes/OCT2017 /train"
val_path="/kaggle/input/oct-classes/OCT2Classes/OCT_2_Classes/OCT2017 /val"
test_path="/kaggle/input/oct-classes/OCT2Classes/OCT_2_Classes/OCT2017 /test"
train_datagen = ImageDataGenerator(
    rescale= 1./255,
    zoom_range= (0.73, 0.9),
    horizontal_flip= True,
    rotation_range= 10,
    width_shift_range= 0.10,
    fill_mode= 'constant',
    height_shift_range= 0.10,   
    brightness_range= (0.55, 0.9)
)
val_datagen = ImageDataGenerator(
    rescale= 1./255, 
)

train_dataset=train_datagen.flow_from_directory(  
    train_path,  
    target_size= (150, 150), 
    color_mode= 'rgb',
    batch_size= 32,  
    class_mode= 'categorical',
    shuffle= True, 
    seed= 1337
)
val_dataset = val_datagen.flow_from_directory(
    val_path,
    target_size= (150, 150),
    color_mode= 'rgb',
    batch_size= 32,  
    class_mode= 'categorical',
    shuffle= True, 
    seed= 1337
)
test_dataset = val_datagen.flow_from_directory(  
    test_path,  
    target_size= (150, 150), 
    color_mode= 'rgb',
    batch_size= 32,        
    class_mode= 'categorical',
    shuffle= False, 
)

Found 64429 images belonging to 2 classes.
Found 26 images belonging to 2 classes.
Found 728 images belonging to 2 classes.


In [5]:
num_classes = len(train_dataset.class_indices)  
train_labels = train_dataset.classes 
train_labels = to_categorical(train_labels, num_classes=num_classes)
valid_labels = val_dataset.classes 
valid_labels = to_categorical(valid_labels, num_classes=num_classes)
nb_train_samples = len(train_dataset.filenames)  
nb_valid_samples = len(val_dataset.filenames)
nb_test_samples = len(test_dataset.filenames)

In [6]:
base_model=keras.models.load_model("/kaggle/input/oct-3classes/finetuned_model.h5")

In [7]:
model=Sequential()
for layer in base_model.layers[:-1]:
    model.add(layer)
model.build(input_shape=(None,150, 150, 3))
for layer in model.layers:
    layer.trainable= False
model.add(Dense(2,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 37, 37, 256)       2

In [8]:
model.compile(optimizer= keras.optimizers.Adam(lr= 0.001), loss= 'categorical_crossentropy', metrics= ['accuracy'])

In [9]:
checkpoint = ModelCheckpoint(
    'baseline_model.h5',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    mode='auto',
    save_weights_only=False,
    period=1
)

earlystop = EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=3,
    verbose=1,
    mode='auto'
)

csvlogger = CSVLogger(
    filename= "baseline_training_csv.log",
    separator = ",",
    append = False
)

reduceLR = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=3,
    verbose=1, 
    mode='auto'
)

callbacks = [checkpoint, earlystop, csvlogger,reduceLR]

In [10]:
history = model.fit(
    train_dataset, 
    epochs = 10,
    steps_per_epoch = nb_train_samples//batch_size,
    validation_data = val_dataset, 
    validation_steps = 1,
    verbose = 2,
    callbacks = callbacks,
    shuffle = True
)

Epoch 1/10


I0000 00:00:1709057300.638166      93 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.



Epoch 1: val_loss improved from inf to 0.02158, saving model to baseline_model.h5
2013/2013 - 1010s - loss: 0.0532 - accuracy: 0.9829 - val_loss: 0.0216 - val_accuracy: 1.0000 - lr: 0.0010 - 1010s/epoch - 502ms/step
Epoch 2/10


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss did not improve from 0.02158
2013/2013 - 557s - loss: 0.0474 - accuracy: 0.9848 - val_loss: 0.0257 - val_accuracy: 1.0000 - lr: 0.0010 - 557s/epoch - 277ms/step
Epoch 3/10

Epoch 3: val_loss did not improve from 0.02158
2013/2013 - 530s - loss: 0.0453 - accuracy: 0.9852 - val_loss: 0.0280 - val_accuracy: 1.0000 - lr: 0.0010 - 530s/epoch - 263ms/step
Epoch 4/10

Epoch 4: val_loss did not improve from 0.02158

Epoch 4: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
2013/2013 - 532s - loss: 0.0447 - accuracy: 0.9856 - val_loss: 0.0309 - val_accuracy: 1.0000 - lr: 0.0010 - 532s/epoch - 264ms/step
Epoch 4: early stopping


In [11]:
(eval_loss, eval_accuracy) = model.evaluate(test_dataset, batch_size = batch_size, verbose= 1)
print('Test Loss: ', eval_loss)
print('Test Accuracy: ', eval_accuracy)

23/23 [==============================] - 9s 418ms/step - loss: 0.0110 - accuracy: 1.0000
Test Loss:  0.010995394550263882
Test Accuracy:  1.0


In [12]:
model.trainable= True
model.compile(optimizer= keras.optimizers.Adam(1e-5), loss= 'categorical_crossentropy', metrics= ['accuracy'])

In [13]:
checkpoint = ModelCheckpoint(
    'finetuned_model.h5',
    monitor='loss',
    verbose=1,
    save_best_only=True,
    mode='auto',
    save_weights_only=False,
    period=1
)

earlystop = EarlyStopping(
    monitor='loss',
    min_delta=0.001,
    patience=3,
    verbose=1,
    mode='auto'
)

csvlogger = CSVLogger(
    filename= "finetuned_training_csv.log",
    separator = ",",
    append = False
)

reduceLR = ReduceLROnPlateau(
    monitor='loss',
    factor=0.1,
    patience=3,
    verbose=1, 
    mode='auto'
)

callbacks = [checkpoint, earlystop, csvlogger,reduceLR]

In [14]:
history_1 = model.fit(
    train_dataset, 
    epochs = 10,
    steps_per_epoch = nb_train_samples//batch_size,
    validation_data = val_dataset, 
    validation_steps = 1,
    verbose = 2,
    callbacks = callbacks,
    shuffle = True
)

Epoch 1/10

Epoch 1: loss improved from inf to 0.05228, saving model to finetuned_model.h5
2013/2013 - 549s - loss: 0.0523 - accuracy: 0.9834 - val_loss: 0.0113 - val_accuracy: 1.0000 - lr: 1.0000e-05 - 549s/epoch - 273ms/step
Epoch 2/10

Epoch 2: loss improved from 0.05228 to 0.04921, saving model to finetuned_model.h5
2013/2013 - 546s - loss: 0.0492 - accuracy: 0.9839 - val_loss: 0.0134 - val_accuracy: 1.0000 - lr: 1.0000e-05 - 546s/epoch - 271ms/step
Epoch 3/10

Epoch 3: loss improved from 0.04921 to 0.04650, saving model to finetuned_model.h5
2013/2013 - 534s - loss: 0.0465 - accuracy: 0.9845 - val_loss: 0.0239 - val_accuracy: 1.0000 - lr: 1.0000e-05 - 534s/epoch - 265ms/step
Epoch 4/10

Epoch 4: loss improved from 0.04650 to 0.04471, saving model to finetuned_model.h5
2013/2013 - 538s - loss: 0.0447 - accuracy: 0.9856 - val_loss: 0.0024 - val_accuracy: 1.0000 - lr: 1.0000e-05 - 538s/epoch - 267ms/step
Epoch 5/10

Epoch 5: loss improved from 0.04471 to 0.04266, saving model to fine

In [15]:
(eval_loss, eval_accuracy) = model.evaluate(test_dataset, batch_size= batch_size, verbose= 1)
print('Test Loss: ', eval_loss)
print('Test Accuracy: ', eval_accuracy)

23/23 [==============================] - 3s 132ms/step - loss: 0.0691 - accuracy: 0.9835
Test Loss:  0.06912650913000107
Test Accuracy:  0.9835164546966553


In [16]:
model.save_weights("Classes_3to2_Model.h5")